In [1]:
import pandas as pd
from pathlib import Path
import tensorflow as tf
from skmultilearn.model_selection import iterative_train_test_split
import numpy as np
import matplotlib.pyplot as plt
import keras_tuner as kt


2025-04-09 23:45:58.922954: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744238758.953631   76465 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744238758.962362   76465 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1744238758.979093   76465 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744238758.979157   76465 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744238758.979159   76465 computation_placer.cc:177] computation placer alr

In [3]:
def set_memory_growth():
    gpus = tf.config.list_physical_devices('GPU')
    if gpus:
      try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
          tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
      except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

def set_memory_limit(memory_limit):
    gpus = tf.config.list_physical_devices('GPU')
    if gpus:
        tf.config.set_logical_device_configuration(
            gpus[0],
            [tf.config.LogicalDeviceConfiguration(memory_limit=memory_limit)]
        )

    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPU,", len(logical_gpus), "Logical GPUs")

set_memory_limit(4096)
#set_memory_growth()
tf.keras.mixed_precision.set_global_policy("mixed_float16")

pd.set_option('display.max_columns', None)

1 Physical GPU, 1 Logical GPUs


In [4]:
df = Path("../histories").glob("mobilenetv2_1.00_224_filtered_multilabel*")
df = [pd.read_csv(csv) for csv in df] 
df = pd.concat(df)
df = df.drop_duplicates(["model", "epoch"], keep="last")
df

,binary_accuracy,f1_score_per_class,f1_score_weighted,loss,val_binary_accuracy,val_f1_score_per_class,val_f1_score_weighted,val_loss,model,epoch,type
0,0.939452,tf.Tensor(\n[0. 0.02298848 0.01503756 ...,0.326755,0.178081,0.968450,tf.Tensor(\n[0. 0. 0.08695652 ...,0.405542,0.106811,mobilenetv2_1.00_224_filtered_multilabel,0,CLF
1,0.970459,tf.Tensor(\n[0. 0. 0.14634144 ...,0.460955,0.097633,0.971706,tf.Tensor(\n[0. 0. 0.08695652 ...,0.490033,0.086371,mobilenetv2_1.00_224_filtered_multilabel,1,CLF
2,0.973165,tf.Tensor(\n[0. 0.01324503 0.31818175 ...,0.524442,0.082583,0.973643,tf.Tensor(\n[0. 0. 0.2222222 ...,0.535161,0.077014,mobilenetv2_1.00_224_filtered_multilabel,2,CLF
3,0.975028,tf.Tensor(\n[0. 0.01333333 0.45180717 ...,0.564923,0.073512,0.975349,tf.Tensor(\n[0. 0. 0.2222222 ...,0.570628,0.070680,mobilenetv2_1.00_224_filtered_multilabel,3,CLF
4,0.976348,tf.Tensor(\n[0. 0.04999998 0.48520705 ...,0.598324,0.067484,0.976357,tf.Tensor(\n[0. 0. 0.28571424 ...,0.595100,0.066293,mobilenetv2_1.00_224_filtered_multilabel,4,CLF
5,0.977705,tf.Tensor(\n[0. 0.10843372 0.52753615 ...,0.630182,0.062001,0.977442,tf.Tensor(\n[0. 0. 0.39999998 ...,0.615522,0.062788,mobilenetv2_1.00_224_filtered_multilabel,5,CLF
6,0.978549,tf.Tensor(\n[0. 0.18604647 0.55932194 ...,0.650010,0.058831,0.978295,tf.Tensor(\n[0. 0.07692306 0.45161283 ...,0.632189,0.060363,mobilenetv2_1.00_224_filtered_multilabel,6,CLF
7,0.979548,tf.Tensor(\n[0. 0.30337074 0.57617724 ...,0.674393,0.054940,0.978837,tf.Tensor(\n[0. 0.14814812 0.51612896 ...,0.644653,0.058310,mobilenetv2_1.00_224_filtered_multilabel,7,CLF
8,0.980120,tf.Tensor(\n[0. 0.31521735 0.6239999 ...,0.690645,0.052350,0.978992,tf.Tensor(\n[0. 0.21428569 0.5454545 ...,0.653462,0.056387,mobilenetv2_1.00_224_filtered_multilabel,8,CLF
9,0.980974,tf.Tensor(\n[0. 0.29189187 0.60752684 ...,0.707402,0.050046,0.980000,tf.Tensor(\n[0. 0.2222222 0.6285714 ...,0.673565,0.054920,mobilenetv2_1.00_224_filtered_multilabel,9,CLF


# Dataset

In [5]:
image_dir = "/home/ruairi/.cache/kagglehub/datasets/bloox2/fieldplant/versions/1/train/"

In [6]:
def decode_img(filename, img_size):
    filepath = str(image_dir) + filename
    img = tf.io.read_file(filepath)
    img = tf.io.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, img_size)
    return img

def process_dataset(img, label):
    img = decode_img(img, (224,224))
    return img, label

In [7]:
df = pd.read_csv("../data/filtered.csv")

X = df.filename.to_frame().to_numpy()
y = df.drop(columns=["filename"]).to_numpy()

X_train, y_train, X_test_val, y_test_val = iterative_train_test_split(X, y, test_size=0.2)
X_test, y_test, X_val, y_val = iterative_train_test_split(X_test_val, y_test_val, test_size=0.5)

train = pd.DataFrame(np.hstack([X_train, y_train]))
val = pd.DataFrame(np.hstack([X_val, y_val]))
test = pd.DataFrame(np.hstack([X_test, y_test]))

train.columns = df.columns
test.columns = df.columns
val.columns = df.columns


In [8]:
BATCH_SIZE=32
epochs = 100
num_classes = len(df.columns[1:])

In [9]:
X_train = train["filename"].values.astype(str)
y_train = train.drop(columns="filename").values.astype("float32")
X_test = test["filename"].values.astype(str)
y_test = test.drop(columns="filename").values.astype("float32")
X_val = val["filename"].values.astype(str)
y_val = val.drop(columns="filename").values.astype("float32")

train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test))
val_ds = tf.data.Dataset.from_tensor_slices((X_val, y_val))

train_ds = train_ds.map(lambda img, label: process_dataset(img, label)).shuffle(buffer_size=train_ds.cardinality()).batch(BATCH_SIZE).cache().prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.map(lambda img, label: process_dataset(img, label)).batch(BATCH_SIZE).cache().prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.map(lambda img, label: process_dataset(img, label)).batch(BATCH_SIZE).cache().prefetch(buffer_size=tf.data.AUTOTUNE)



# Model

In [ ]:
preprocessing = tf.keras.applications.mobilenet_v2.preprocess_input
base_model = tf.keras.applications.mobilenet_v2.MobileNetV2(include_top=False, weights="imagenet", input_shape=(224, 224, 3))
base_model.trainable = False

inputs = tf.keras.Input(shape=(224, 224, 3))
x = preprocessing(inputs)
x = base_model(x, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.5)(x)
outputs = tf.keras.layers.Dense(num_classes, activation="sigmoid")(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(
    optimizer = tf.keras.optimizers.Adam(),
    loss="binary_crossentropy",
    metrics = [
        "binary_accuracy",
        tf.keras.metrics.F1Score(average="weighted", threshold=0.5, name="f1_score_weighted", dtype=None),
        tf.keras.metrics.F1Score(average=None, threshold=0.5, name="f1_score_per_class", dtype=None)
    ]
)

cbs = [
    tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True, verbose=1),
    tf.keras.callbacks.ModelCheckpoint(filepath="../saved_models/final_mn.keras", save_best_only=True, monitor="val_loss", verbose=1)
]

model.summary(show_trainable=True, expand_nested=True)

In [ ]:
history = model.fit(train_ds, validation_data=val_ds,
                    epochs=epochs, callbacks=cbs)
history_df = pd.DataFrame(history.history)
history_df['epoch'] = history.epoch
history_df['type'] = "CLF"

### Finetuning

In [ ]:
best_epoch = history_df.iloc[history_df.val_loss.idxmin()]
best_val_loss = best_epoch.val_loss
best_val_loss

In [ ]:
base_model = model.get_layer(name="mobilenetv2_1.00_224")
freeze_layer = base_model.get_layer(name="block_14_expand")
freeze_num = 0
for i, layer in enumerate(base_model.layers):
    if layer == freeze_layer:
        freeze_num = i

In [ ]:
for layer in base_model.layers[:freeze_num]:
    layer.trainable = False
for layer in base_model.layers[freeze_num:]:
    layer.trainable = True

lr = model.get_compile_config()['optimizer']['config']['learning_rate'] / 10

model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr),
    loss="binary_crossentropy",
    metrics = [
        "binary_accuracy",
        tf.keras.metrics.F1Score(average="weighted", threshold=0.5, name="f1_score_weighted", dtype=None),
        tf.keras.metrics.F1Score(average=None, threshold=0.5, name="f1_score_per_class", dtype=None)
    ]
)

cbs = [
    tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True, verbose=1, baseline=best_val_loss),
    tf.keras.callbacks.ModelCheckpoint(filepath="../saved_models/final_mn.keras", save_best_only=True, monitor="val_loss", verbose=1, initial_value_threshold=best_val_loss)
]

model.summary(show_trainable=True, expand_nested=True)

In [ ]:
history_ft = model.fit(train_ds, validation_data=val_ds,
                    epochs=epochs, callbacks=cbs, initial_epoch=best_epoch.epoch+1)

hist_ft_df = pd.DataFrame(history_ft.history)
hist_ft_df['epoch'] = history_ft.epoch
hist_ft_df['type'] = "FT"

In [ ]:
hist_df = pd.concat([history_df, hist_ft_df])
hist_df = hist_df.drop_duplicates(subset=["epoch"], keep="last")

plt.plot(history_df.epoch, history_df.val_loss)
plt.plot(hist_ft_df.epoch, hist_ft_df.val_loss)

# Hyperparameter Tuning

In [10]:
def call_existing_code(units, dropout_rate):
    base_model = tf.keras.applications.MobileNetV2(include_top=False, input_shape=(224,224,3))
    base_model.trainable = False

    inputs = tf.keras.Input(shape=base_model.input_shape[1:])
    x = tf.keras.applications.mobilenet_v2.preprocess_input(inputs)
    x = base_model(x, training=False)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    
    if units != 0:
        x = tf.keras.layers.Dense(units=units, use_bias=False)(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation("relu")(x)
        x = tf.keras.layers.Dropout(0.5)(x)

        
    outputs = tf.keras.layers.Dense(num_classes, activation="sigmoid", name="clasifier")(x)
    model = tf.keras.Model(inputs, outputs)

    model.compile(
        optimizer= tf.keras.optimizers.Adam(learning_rate=0.0001),
        loss="binary_crossentropy",
        metrics=["binary_accuracy",
                 tf.keras.metrics.F1Score(average="weighted", threshold=0.5, name="f1_score_weighted", dtype=None),
                 tf.keras.metrics.F1Score(average=None, threshold=0.5, name="f1_score_per_class", dtype=None)]
        )
    return model

def build_model(hp):
    units = hp.Choice("units", [0, 256, 512])
    dropout_rate = hp.Choice("dropout", [0.2, 0.5])

    model = call_existing_code(
        units=units, dropout_rate=dropout_rate
    )
    return model

In [11]:
tuner = kt.RandomSearch(
    hypermodel= build_model,
    objective = kt.Objective("val_loss", "min"),
    directory="../hyperparameterTuning/",
    project_name="mobilenet_HT",
    overwrite=False,
    max_trials=40,
    seed=1
)
tuner.search_space_summary()

Search space summary
Default search space size: 2
units (Choice)
{'default': 0, 'conditions': [], 'values': [0, 256, 512], 'ordered': True}
dropout (Choice)
{'default': 0.2, 'conditions': [], 'values': [0.2, 0.5], 'ordered': True}


In [12]:
cbs = [tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True, verbose=1)]

tuner.search(train_ds, validation_data=val_ds, epochs=epochs, callbacks=cbs, verbose=1)

Trial 1 Complete [00h 10m 13s]
val_loss: 0.05155941843986511

Best val_loss So Far: 0.05155941843986511
Total elapsed time: 00h 10m 13s

Search: Running Trial #2

Value             |Best Value So Far |Hyperparameter
0                 |0                 |units
0.2               |0.5               |dropout

Epoch 1/100
129/129 ━━━━━━━━━━━━━━━━━━━━ 17s 90ms/step - binary_accuracy: 0.7340 - f1_score_per_class: 0.0616 - f1_score_weighted: 0.2029 - loss: 0.5396 - val_binary_accuracy: 0.9594 - val_f1_score_per_class: 0.0200 - val_f1_score_weighted: 0.0984 - val_loss: 0.1470
Epoch 2/100
129/129 ━━━━━━━━━━━━━━━━━━━━ 6s 46ms/step - binary_accuracy: 0.9549 - f1_score_per_class: 0.0579 - f1_score_weighted: 0.2295 - loss: 0.1648 - val_binary_accuracy: 0.9622 - val_f1_score_per_class: 0.0499 - val_f1_score_weighted: 0.2417 - val_loss: 0.1215
Epoch 3/100
129/129 ━━━━━━━━━━━━━━━━━━━━ 6s 45ms/step - binary_accuracy: 0.9622 - f1_score_per_class: 0.0677 - f1_score_weighted: 0.3142 - loss: 0.1362 - val_bi

KeyboardInterrupt: 

In [ ]:
tuner.results_summary(32)